In [1]:
#Embedding generation using emb_dataloader which checks the path and verifies the file type and images

In [2]:
# some packages need async support in jupyter
import nest_asyncio
nest_asyncio.apply()

In [3]:
# layer_name can also be given 
# for name, module in model.named_modules():
#     if isinstance(module, torch.nn.AdaptiveAvgPool2d):  # Look for avgpool layers
#         print(name, module)
# output:
# avgpool AdaptiveAvgPool2d(output_size=(1, 1))
# layer4.2.downsample.1 AdaptiveAvgPool2d(output_size=(1, 1))

## layer_name = 'self.model.layer4[2].downsample[1]' or layer='avgpool'

In [4]:
from mb_pytorch.models.modelloader import ModelLoader
from mb_pytorch.utils.yaml_reader import YAMLReader
from mb_utils.src.logging import logger

In [5]:
yaml_read = YAMLReader('../scripts/loader_y.yaml') #load the data
yaml_data = yaml_read.read()

In [6]:
yaml_data['model']

{'model_type': 'detection',
 'model': 'faster_rcnn',
 'use_pretrained': True,
 'use_own_model': False,
 'model_path': '/home/malav/Desktop/mb_packages/mb_pytorch/models/...',
 'device': 'cpu',
 'load_model': False,
 'use_torchvision_models': True,
 'model_name': 'fasterrcnn_resnet50_fpn',
 'model_version': '50',
 'model_backbone': 'resnet',
 'model_optimizer': 'Adam',
 'model_loss': 'cross_entropy',
 'model_scheduler': 'StepLR',
 'model_pretrained': True,
 'model_epochs': 3,
 'model_num_classes': 41,
 'model_initialization': 'None',
 'model_meta_data': {'model_bbox_threshold': 0.8},
 'model_train_parameters': {'Adam': {'lr': 0.001,
   'betas': [0.9, 0.999],
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False},
  'SGD': {'lr': 0.01,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False},
  'RMSprop': {'lr': 0.01,
   'alpha': 0.99,
   'eps': 1e-08,
   'weight_decay': 0,
   'momentum': 0,
   'centered': False},
  'cross_entropy': {'weight': 'None',
   's

In [7]:
ModelLoader?

Init signature: ModelLoader(data: dict, logger=None)
Docstring:     
Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            return F.relu(self.conv2(x))

Submodules assigned in this way will be registered, and will have their
parameters converted too when you call :meth:`to`, etc.

.. note::
    As per the example above, an ``__init__()`` call to the parent class
    must be made before assignment on the child.

:ivar training: Boolean represents whether this module is in training or
                

In [8]:
model_init = ModelLoader(yaml_data['model'])

In [9]:
model = model_init.get_model()

/home/malav/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/malav/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Thu 06:16:43,442 INF Model fasterrcnn_resnet50_fpn loaded from torchvision.models.                                                   


In [10]:
from mb_pytorch.dataloader.loader import BaseDataset, TorchDataLoader
train_dataset = BaseDataset(data_config=yaml_data['data']['file'], task_type=yaml_data['model']['model_type'], 
                            transform=yaml_data['transformation'],is_train=True,logger=logger)
train_dataloader= TorchDataLoader(dataset=train_dataset, batch_size=yaml_data['train_params']['batch_size'],
                                  shuffle=yaml_data['train_params']['shuffle'],num_workers=yaml_data['train_params']['num_workers'])

Thu 06:16:44,130 INF Loading data from /home/malav/Desktop/mb_packages/mb_pytorch/data/fruit_dataset/fruit_dataset_final.csv         
Thu 06:16:44,132 INF Loading DataFrame from /home/malav/Desktop/mb_packages/mb_pytorch/data/fruit_dataset/fruit_dataset_final.csv    
Thu 06:16:44,136 INF Successfully loaded DataFrame with shape (300, 5)                                                               
Thu 06:16:44,140 INF Checking duplicates for columns: ['image_path']                                                                 
Thu 06:16:44,143 INF No duplicates found                                                                                             
Thu 06:16:44,145 INF Removing unnamed columns                                                                                        


In [11]:
from mb_pytorch.models.modelloader import LayerExtractor

In [12]:
LayerExtractor?

Init signature: LayerExtractor(model, dataloader, target_layers: list, device='cpu')
Docstring:      <no docstring>
Init docstring:
Initialize the LayerExtractor class.

Args:
    model (nn.Module): Model which we want to extract the features from.
    dataloader (DataLoader): Dataloader which will be used to extract the features.
    target_layers (list): List of target layers from which the features are to be extracted.
    device (str, optional): Device on which the model is to be run. Defaults to 'cpu'.
File:           ~/.local/lib/python3.11/site-packages/mb_pytorch/models/modelloader.py
Type:           type
Subclasses:     

In [ ]:
emb_init = LayerExtractor(model,dataloader=train_dataloader,target_layers=['backbone.body.layer3.1'],device='cpu')

In [20]:
emb_data = emb_init.generate_embeddings()

30it [05:26, 10.87s/it]


In [21]:
emb_data

{'backbone.body.layer3.1': [array([[[0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.29735452, 0.19107917, 0.25537887, ..., 0.24141978,
           0.33107916, 0.18637455],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          ...,
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.00493136, 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ]],
  
         [[0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.38545454, 0.5019762 , 0.5801352 , ..., 0.2414197 ,
           0.33107865, 0.18637466],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          ...,
          [0.        , 0.        , 0.   

In [18]:
for n,m in model.named_modules():
    print(n)


transform
backbone
backbone.body
backbone.body.conv1
backbone.body.bn1
backbone.body.relu
backbone.body.maxpool
backbone.body.layer1
backbone.body.layer1.0
backbone.body.layer1.0.conv1
backbone.body.layer1.0.bn1
backbone.body.layer1.0.conv2
backbone.body.layer1.0.bn2
backbone.body.layer1.0.conv3
backbone.body.layer1.0.bn3
backbone.body.layer1.0.relu
backbone.body.layer1.0.downsample
backbone.body.layer1.0.downsample.0
backbone.body.layer1.0.downsample.1
backbone.body.layer1.1
backbone.body.layer1.1.conv1
backbone.body.layer1.1.bn1
backbone.body.layer1.1.conv2
backbone.body.layer1.1.bn2
backbone.body.layer1.1.conv3
backbone.body.layer1.1.bn3
backbone.body.layer1.1.relu
backbone.body.layer1.2
backbone.body.layer1.2.conv1
backbone.body.layer1.2.bn1
backbone.body.layer1.2.conv2
backbone.body.layer1.2.bn2
backbone.body.layer1.2.conv3
backbone.body.layer1.2.bn3
backbone.body.layer1.2.relu
backbone.body.layer2
backbone.body.layer2.0
backbone.body.layer2.0.conv1
backbone.body.layer2.0.bn1
bac

In [ ]:
from tqdm import tqdm
class LayerExtractor2:
    def __init__(self, model, dataloader, target_layers : list ,device='cpu'):
        """
        Initialize the LayerExtractor class.

        Args:
            model (nn.Module): Model which we want to extract the features from.
            dataloader (DataLoader): Dataloader which will be used to extract the features.
            target_layers (list): List of target layers from which the features are to be extracted.
            device (str, optional): Device on which the model is to be run. Defaults to 'cpu'.
        """
        self.model = model
        self.dataloader = dataloader
        self.target_layers = target_layers
        self.device = device
        self._features = {}
        
        model_layers = dict(self.model.named_modules())
        for layer in self.target_layers:
            if layer not in model_layers:
                raise ValueError(f"Layer '{layer}' not found in the model")

        for name, module in self.model.named_modules():
            if name in self.target_layers:
                module.register_forward_hook(self._hook_fn(name))

    def _hook_fn(self, layer_name):
        """
        Creates a forward hook function for a specified layer.

        This hook function captures the output of the layer during the forward pass
        and stores it in a dictionary with the layer name as the key. The output
        is reshaped and converted to a NumPy array before storing.

        Args:
            layer_name (str): The name of the layer to attach the forward hook to.

        Returns:
            function: A hook function that can be registered to a model layer.
        """

        def hook(module, input, output):
            N, C, H, W = output.shape
            output = output.reshape(N, C, -1)
            self._features[layer_name]=output.data.cpu().numpy()
        return hook

    def forward(self, x):
        x = x.to(self.device)
        _ = self.model(x)  # Run the forward pass
        return self._features  # Return the hooked layer's features
    
    def generate_embeddings(self):
        self.model.to(self.device)
        self.model.eval()

        final_features = {i_layer: [] for i_layer in self.target_layers}

        for i, i_dat in tqdm(enumerate(self.dataloader)):
            # Reset _features for each forward pass
            self._features = {i_layer: None for i_layer in self.target_layers}
            
            # Forward pass
            _ = self.model(i_dat['image'])
            
            # Append features for each layer
            for layer_name in self.target_layers:
                final_features[layer_name].append(self._features[layer_name])

        return final_features